# BQ Compression – Load Data and compress vectors

## Get keys and urls

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_HTTP_URL = os.getenv("WEAVIATE_HTTP_URL")
WEAVIATE_GRPC_URL = os.getenv("WEAVIATE_GRPC_URL")
AZURE_BASE_URL = os.getenv("AZURE_BASE_URL")
AZURE_RESOURCE_NAME = os.getenv("AZURE_RESOURCE_NAME")

print(WEAVIATE_HTTP_URL)
print(WEAVIATE_GRPC_URL)
print(AZURE_BASE_URL)
print(AZURE_RESOURCE_NAME)

## Connect to Weaviate

In [ ]:
import weaviate
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_HTTP_URL,
    http_port=80,
    http_secure=False,
    grpc_host=WEAVIATE_GRPC_URL,
    grpc_port=50051,
    grpc_secure=False,
)

client.is_ready()

## Create Collection with BQ configuration

[Docs: Binary Quantization (BQ)](https://weaviate.io/developers/weaviate/configuration/compression/bq-compression)

Note #1: Binary Quantization works from the first object added to the collection. No training required.

Note #2: Binary Quantization works both with HNSW and Flat index.

In [ ]:
from weaviate.classes.config import Configure, VectorDistances

client.collections.delete("WikiBQ")

# Create a collection here - with Cohere as a vectorizer
client.collections.create(
    name="WikiBQ",

    vectorizer_config=[
        Configure.NamedVectors.text2vec_openai(
            name="main_vector",

            deployment_id="text-embedding-3-small",
            base_url=AZURE_BASE_URL,

            source_properties=['title', 'text'],

            # Configure BQ with flat vector index
            vector_index_config=Configure.VectorIndex.flat(
                distance_metric=VectorDistances.COSINE,
                vector_cache_max_objects=100_000,
                quantizer=Configure.VectorIndex.Quantizer.bq(
                    rescore_limit=200,
                    cache=True
                )
            ),

            # HSNW example
            # vector_index_config=Configure.VectorIndex.hsnw(
            #     quantizer=Configure.VectorIndex.Quantizer.bq(
            #         rescore_limit=200,
            #         cache=True
            #     )
            # ),
        )
    ],
)

## The rest is the same

In [ ]:
from data_loader import import_wiki_data
import_wiki_data(client, "WikiBQ", 25000)

In [ ]:
wikiBQ = client.collections.get("WikiBQ")
wikiBQ.aggregate.over_all()

## Clean up

In [ ]:
# client.collections.delete("WikiBQ")

## Close the client

In [ ]:
client.close()